In [1]:
import numpy as np
import pandas as pd

from tqdm import tqdm

### 数据预处理

In [2]:
# 导入价差数据
spread = pd.read_stata('.\\AMQI\\raw\\spread.dta')
spread.head()

,证券代码,交易日期,相对报价价差等权平均,相对有效价差等权平均,相对报价价差时间加权平均,相对有效价差时间加权平均,相对报价价差成交量加权平均,相对有效价差成交量加权平均,相对报价价差成交金额加权平均,相对有效价差成交金额加权平均,有效数据条数
0,000001,2003-01-02,0.120330,0.119181,0.125846,0.124423,0.133102,0.132468,0.133072,0.132434,1030
1,000001,2003-01-03,0.114652,0.113188,0.117521,0.115589,0.129906,0.129329,0.129889,0.129312,1094
2,000001,2003-01-06,0.114064,0.112572,0.117581,0.115452,0.122045,0.121641,0.122073,0.121669,937
3,000001,2003-01-07,0.110642,0.109317,0.114525,0.112769,0.113870,0.113490,0.113900,0.113521,898
4,000001,2003-01-08,0.132515,0.130481,0.143690,0.141348,0.153468,0.151170,0.153872,0.151594,680


In [3]:
%%time
# 数据预处理
spread1 = spread[['证券代码','交易日期','相对报价价差等权平均','相对报价价差时间加权平均']]
#spread1 = spread[['证券代码','交易日期','相对报价价差成交量加权平均','相对报价价差成交金额加权平均']]
spread1 = spread1.rename(columns={spread1.columns[0]:'code',
                                  spread1.columns[1]:'date',
                                  spread1.columns[2]:'qsba',
                                  spread1.columns[3]:'esba'})
spread1['date'] = pd.to_datetime(spread1['date'])
spread1['year'] = spread1.date.dt.year
spread1['month'] = spread1.date.dt.month

CPU times: total: 4.66 s
Wall time: 17.1 s


In [4]:
spread1[spread1.year==2024]

,code,date,qsba,esba,year,month
5855124,000001,2024-01-02,0.108075,0.106751,2024,1
5855125,000001,2024-01-03,0.109022,0.109017,2024,1
5855126,000001,2024-01-04,0.110292,0.110263,2024,1
5855127,000001,2024-01-05,0.108132,0.106805,2024,1
5855128,000001,2024-01-08,0.109926,0.109903,2024,1
...,...,...,...,...,...,...
12383926,689009,2024-02-23,0.068951,0.070510,2024,2
12383927,689009,2024-02-26,0.118275,0.118249,2024,2
12383928,689009,2024-02-27,0.080811,0.083587,2024,2
12383929,689009,2024-02-28,0.077991,0.081080,2024,2


In [5]:
4976/21

236.95238095238096

In [6]:
%%time
# 构造分组指标
spread2 = spread1.copy()
spread2 = spread2.sort_values(by='date')
#spread2['last_date'] = np.where((spread2.groupby('code')['month'].diff(-1) == 0),
#                                pd.NaT,
#                                spread2['date'])
#spread2['last_date'] = spread2.groupby(['code','year','month'])['last_date'].bfill()
spread2 = spread2.groupby(['code','year','month'])[['qsba','esba']].mean().reset_index()

# 调整日期格式
#spread2['last_date'] = pd.to_datetime(spread2['last_date'])
#spread2 = spread2.rename(columns={'last_date':'date'})

CPU times: total: 5.2 s
Wall time: 11.7 s


In [7]:
spread2[(spread2.code=='000001') & (spread2.year==2007)]

,code,year,month,qsba,esba
48,000001,2007,1,0.096785,0.095514
49,000001,2007,2,0.094131,0.094842
50,000001,2007,3,0.082538,0.083875
51,000001,2007,4,0.073668,0.074533
52,000001,2007,5,0.083294,0.079997
53,000001,2007,6,0.088898,0.087200
54,000001,2007,7,0.072205,0.073559
55,000001,2007,8,0.075758,0.077993
56,000001,2007,9,0.067096,0.069528
57,000001,2007,10,0.077869,0.080930


In [8]:
# 导入价格数据进行合并
price = pd.read_pickle('.\\AMQI\\processed\\price_month.pkl')
spread3 = spread2.copy()
#spread3['year'] = spread3.date.dt.year
#spread3['month'] = spread3.date.dt.month
#spread3 = spread3.drop('date',axis=1)
price = pd.merge(price,spread3,
                 how='left',on=['code','year','month'])
price = price[(price.date >= pd.to_datetime('20070601'))] # 确定时间

#price['qsba'] = np.where(price.close_price==0,np.nan,price.qsba)
#price['esba'] = np.where(price.close_price==0,np.nan,price.esba)
print('空缺值：\n',price.isna().sum()) # 缺失值有两种情况：停牌 / 缺失数据

price.head()

空缺值：
 code                   0
date                   0
close_price            0
market_value           0
return                 0
month                  0
year                   0
market                 0
total_value            0
risk_free_month        0
qsba               20961
esba               20965
dtype: int64


,code,date,close_price,market_value,return,month,year,market,total_value,risk_free_month,qsba,esba
193,000001,2007-06-29,27.520000,42661168.0,-0.040781,6,2007,-0.057379,5.742759e+07,0.25150,0.088898,0.087200
194,000001,2007-07-31,36.230000,56163304.0,0.316497,7,2007,0.206212,7.560325e+07,0.26245,0.072205,0.073559
195,000001,2007-08-31,38.000000,58907136.0,0.048855,8,2007,0.189608,7.929682e+07,0.28430,0.075758,0.077993
196,000001,2007-09-28,39.980000,61976508.0,0.052105,9,2007,0.078187,8.342860e+07,0.30605,0.067096,0.069528
197,000001,2007-10-31,48.049999,74486520.0,0.201851,10,2007,0.015500,1.002687e+08,0.31690,0.077869,0.080930


In [9]:
# 进行分组
# 导入主板数据并进行预处理
main = pd.read_excel('.\\AMQI\\raw\\EDE20241125.xlsx').iloc[:-2,[2]]
main = main.rename(columns={main.columns[0]:'code'})
main['code'] = main['code'].astype(int).astype(str).str.zfill(6)
main['if_mainboard'] = 1

# 合并数据
price1 = pd.merge(price,main,
                  how='left',on='code')
price1['if_mainboard'] = price1['if_mainboard'].fillna(0)
price1.head()

,code,date,close_price,market_value,return,month,year,market,total_value,risk_free_month,qsba,esba,if_mainboard
0,000001,2007-06-29,27.520000,42661168.0,-0.040781,6,2007,-0.057379,5.742759e+07,0.25150,0.088898,0.087200,1.0
1,000001,2007-07-31,36.230000,56163304.0,0.316497,7,2007,0.206212,7.560325e+07,0.26245,0.072205,0.073559,1.0
2,000001,2007-08-31,38.000000,58907136.0,0.048855,8,2007,0.189608,7.929682e+07,0.28430,0.075758,0.077993,1.0
3,000001,2007-09-28,39.980000,61976508.0,0.052105,9,2007,0.078187,8.342860e+07,0.30605,0.067096,0.069528,1.0
4,000001,2007-10-31,48.049999,74486520.0,0.201851,10,2007,0.015500,1.002687e+08,0.31690,0.077869,0.080930,1.0


In [68]:
def portfolio_sort(df,sort_col,g_num,hold_m=1,switch=False):
    # 上个月的数据用于下个月的分组
    df = df.sort_values(by=['date','code']) # 保证次序正确
    df['to_sort'] = df.groupby('code')[sort_col].shift(1)
    #df = df.dropna(subset=['to_sort']) # 是np.nan的值不参与portfolio的构造
    #df = df[df['close_price']!=0] # drop掉停牌数据

    # 根据主板计算分位数
    group = [i/g_num for i in range(1,g_num)]
    quantile = df[df['if_mainboard'] == 1].groupby(['year','month'])['to_sort'].apply(
        lambda x:x.quantile(group)).reset_index().rename(columns={'level_2':'group'}).dropna()
    quantile['group'] = quantile['group']/(1/g_num) # 调整为对应组号
    quantile['group'] = quantile['group'].map(lambda x:round(x,0))
    quantile1 = quantile.pivot(index=['year', 'month'], columns='group', values='to_sort').reset_index() # 转换为宽格式
    df1 = pd.merge(df,quantile1,
                   how='inner',
                   on=['year','month']) # 将分组依据合并

    # 进行分组
    df1['portfolio0'] = np.nan # 用来存储分组
    for i in tqdm(range(1,g_num)):
        df1['portfolio0'] = np.where(((df1['portfolio0'].isna()) & (df1['to_sort'] <= df1[i])), # 如果尚未填充且小于分位数，说明分组应为本轮循环对应组数
                                    i,df1['portfolio0'])
    df1['portfolio0'] = np.where(((df1['portfolio0'].isna()) & (df1['to_sort'] > df1[g_num-1])),g_num,df1['portfolio0'])


    risk_free = df1.groupby(['year','month'])['risk_free_month'].mean().reset_index()
    risk_free['risk_free_month'] = risk_free['risk_free_month']*0.01 # 调整无风险利率单位错误

    for i in tqdm(range(1,hold_m)):
        df1['portfolio'+str(i)] = df1.groupby('code')['portfolio0'].shift(i)
     
    indicator = df1[['code','date','return',sort_col]]
    
    if hold_m > 1:
        indicator = indicator.rename(columns={sort_col:sort_col+str(0)})
        for i in tqdm(range(1,hold_m)):
            indicator[sort_col+str(i)] = indicator.groupby('code')[sort_col+str(0)].shift(i)
        indicator[sort_col] = indicator[[sort_col+str(i) for i in range(hold_m)]].mean(axis=1)
        indicator.drop([sort_col+str(i) for i in range(hold_m)],axis=1,inplace=True)
    
    indicator = indicator.sort_values(by='date')
    indicator['year'] = indicator.date.dt.year
    indicator['month'] = indicator.date.dt.month
    indicator['year_month'] = indicator['year'].astype(str) + '-' + indicator['month'].astype(str).str.zfill(2)
    indicator.drop(['date','month','year'],axis=1,inplace=True)
    
    indicator = indicator.rename(columns={sort_col:'indicator_value',
                                          'return':'monthly_return'})

    def port_return(W,risk_free):
        W = W.pivot(index=['year','month'],columns='portfolio',values='return').reset_index()
        if switch:
            W = W.rename(columns={1:'long',5:'short'})[['year','month','long','short']]
        else:
            W = W.rename(columns={5:'long',1:'short'})[['year','month','long','short']]
        W = pd.merge(W,risk_free,
                     how='left',on=['year','month']) # 粘上无风险利率
        W['long'] = W['long'] - W['risk_free_month']
        W['short'] = W['short'] - W['risk_free_month']
        W['spread'] = W['long'] - W['short']
        return(W)
    
    # 计算有多期持有的情况
    def holding(W,hold_m,weighted=False):
        for i in tqdm(range(1,hold_m)):
            if weighted:
                W_new = df1.groupby(['month','year','portfolio'+str(i)]).apply(
                    lambda x:(x['return']*x['market_value']).sum() / x['market_value'].sum()).reset_index(
                ).rename(columns={0:'return'+str(i),
                                  'portfolio'+str(i):'portfolio0'})
            else:
                W_new = df1.groupby(['month','year','portfolio'+str(i)])['return'].mean().reset_index(
                ).rename(columns={'return':'return'+str(i),
                                  'portfolio'+str(i):'portfolio0'})

            W = pd.merge(W,W_new,
                         on=['month','year','portfolio0'])
        W['return'] = W[['return'+str(i) for i in range(hold_m)]].mean(axis=1)
        W = W[['month','year','portfolio0','return']].rename(columns={'portfolio0':'portfolio'})
        return W

    # 计算平均收益率
    EW = df1.groupby(['month','year','portfolio0'])['return'].mean().reset_index().rename(columns={'return':'return0'})
    EW = EW.sort_values(by=['year','month','portfolio0'])
    EW = holding(EW,hold_m)
    EW = port_return(EW,risk_free)
    
    # 计算加权收益率
    VW = df1.groupby(['month', 'year', 'portfolio0']).apply(
        lambda x:(x['return']*x['market_value']).sum() / x['market_value'].sum()).reset_index().rename(columns={0:'return0'})
    VW = VW.sort_values(by=['year','month','portfolio0'])
    VW = holding(VW,hold_m,True)
    VW = port_return(VW,risk_free)
    
    # 调整格式为统一格式
    EW['year_month'] = EW['year'].astype(str) + '-' + EW['month'].astype(str).str.zfill(2)
    EW = EW.drop(['year','month'],axis=1)
    EW = EW.rename(columns={'long':'return_long_eq',
                            'short':'return_short_eq',
                            'spread':'return_spread_eq'})
    VW['year_month'] = VW['year'].astype(str) + '-' + VW['month'].astype(str).str.zfill(2)
    VW = VW.drop(['year','month'],axis=1) 
    VW = VW.rename(columns={'long':'return_long_mv',
                            'short':'return_short_mv',
                            'spread':'return_spread_mv'})
    
    result = pd.merge(EW,VW,on=['year_month','risk_free_month'])
    
    return indicator,result

1个月

In [33]:
# 根据 QSBA 计算分组
sort_col = 'qsba'
df = price1.copy()
g_num = 5
hold_m = 1

qsba_ind, qsba = portfolio_sort(df,sort_col,g_num)
qsba_ind.to_csv('.\\AMQI\\output\\indicator\\qsba_indicators.csv')
qsba.to_csv('.\\AMQI\\output\\return\\qsba.csv')

# 根据 ESBA 计算分组
sort_col = 'esba'
df = price1.copy()
g_num = 5
hold_m = 1

esba_ind, esba = portfolio_sort(df,sort_col,g_num)
esba_ind.to_csv('.\\AMQI\\output\\indicator\\esba_indicators.csv')
esba.to_csv('.\\AMQI\\output\\return\\esba.csv')

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 93.05it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 96.94it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [34]:
esba.drop('year_month',axis=1).mean()

return_long_eq      0.010739
return_short_eq    -0.011507
risk_free_month     0.001843
return_spread_eq    0.022246
return_long_mv      0.011628
return_short_mv     0.002990
return_spread_mv    0.008638
dtype: float64

3个月

In [69]:
# 根据 QSBA 计算分组
sort_col = 'qsba'
df = price1.copy()
g_num = 5
hold_m = 3

qsba_ind, qsba = portfolio_sort(df,sort_col,g_num,hold_m)
qsba_ind.to_csv('.\\AMQI\\output\\indicator\\qsba3_indicators.csv')
qsba.to_csv('.\\AMQI\\output\\return\\qsba3.csv')

# 根据 ESBA 计算分组
sort_col = 'esba'
df = price1.copy()
g_num = 5
hold_m = 3

esba_ind, esba = portfolio_sort(df,sort_col,g_num,hold_m)
esba_ind.to_csv('.\\AMQI\\output\\indicator\\esba3_indicators.csv')
esba.to_csv('.\\AMQI\\output\\return\\esba3.csv')

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.02s/it]


In [70]:
qsba.drop('year_month',axis=1).mean()

return_long_eq      0.008176
return_short_eq    -0.010493
risk_free_month     0.001834
return_spread_eq    0.018670
return_long_mv      0.009153
return_short_mv     0.001793
return_spread_mv    0.007360
dtype: float64

6个月

In [71]:
# 根据 QSBA 计算分组
sort_col = 'qsba'
df = price1.copy()
g_num = 5
hold_m = 6

qsba_ind, qsba = portfolio_sort(df,sort_col,g_num,hold_m)
qsba_ind.to_csv('.\\AMQI\\output\\indicator\\qsba6_indicators.csv')
qsba.to_csv('.\\AMQI\\output\\return\\qsba6.csv')

# 根据 ESBA 计算分组
sort_col = 'esba'
df = price1.copy()
g_num = 5
hold_m = 6

esba_ind, esba = portfolio_sort(df,sort_col,g_num,hold_m)
esba_ind.to_csv('.\\AMQI\\output\\indicator\\esba6_indicators.csv')
esba.to_csv('.\\AMQI\\output\\return\\esba6.csv')

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.30it/s]


9个月

In [72]:
# 根据 QSBA 计算分组
sort_col = 'qsba'
df = price1.copy()
g_num = 5
hold_m = 9

qsba_ind, qsba = portfolio_sort(df,sort_col,g_num,hold_m)
qsba_ind.to_csv('.\\AMQI\\output\\indicator\\qsba9_indicators.csv')
qsba.to_csv('.\\AMQI\\output\\return\\qsba9.csv')

# 根据 ESBA 计算分组
sort_col = 'esba'
df = price1.copy()
g_num = 5
hold_m = 9

esba_ind, esba = portfolio_sort(df,sort_col,g_num,hold_m)
esba_ind.to_csv('.\\AMQI\\output\\indicator\\esba9_indicators.csv')
esba.to_csv('.\\AMQI\\output\\return\\esba9.csv')

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:07<00:00,  1.11it/s]


12个月

In [73]:
# 根据 QSBA 计算分组
sort_col = 'qsba'
df = price1.copy()
g_num = 5
hold_m = 12

qsba_ind, qsba = portfolio_sort(df,sort_col,g_num,hold_m)
qsba_ind.to_csv('.\\AMQI\\output\\indicator\\qsba12_indicators.csv')
qsba.to_csv('.\\AMQI\\output\\return\\qsba12.csv')

# 根据 ESBA 计算分组
sort_col = 'esba'
df = price1.copy()
g_num = 5
hold_m = 12

esba_ind, esba = portfolio_sort(df,sort_col,g_num,hold_m)
esba_ind.to_csv('.\\AMQI\\output\\indicator\\esba12_indicators.csv')
esba.to_csv('.\\AMQI\\output\\return\\esba12.csv')

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:10<00:00,  1.05it/s]


In [74]:
esba.drop('year_month',axis=1).mean()

return_long_eq      0.008269
return_short_eq    -0.005394
risk_free_month     0.001764
return_spread_eq    0.013663
return_long_mv      0.010836
return_short_mv     0.004868
return_spread_mv    0.005968
dtype: float64

### 其他因子

#### rtv

In [75]:
# 价格数据
price2 = price1.copy()
price2 = price2.drop(['qsba','esba'],axis=1)

# 合并indicator
rtv = pd.read_csv('.\\AMQI\\processed\\rtv1.csv')
rtv = rtv.rename(columns={rtv.columns[0]:'code',
                          rtv.columns[1]:'date',
                          rtv.columns[2]:'rtv'})
rtv['date'] = pd.to_datetime(rtv['date'])
rtv['code'] = rtv['code'].astype(str).str.zfill(6)
rtv['year'] = rtv.date.dt.year
rtv['month'] = rtv.date.dt.month
rtv = rtv.drop('date',axis=1)

price2 = pd.merge(price2,rtv,on=['code','year','month'],how='left')
price2 = price2[(price2.date >= pd.to_datetime('20070601'))] # 确定时间

In [76]:
# 根据 rtv 计算分组
# 1月
sort_col = 'rtv'
df = price2.copy()
g_num = 5
hold_m = 1

rtv_ind, rtv = portfolio_sort(df,sort_col,g_num,hold_m,True)
rtv_ind.to_csv('.\\AMQI\\output\\indicator\\rtv_indicators.csv')
rtv.to_csv('.\\AMQI\\output\\return\\rtv.csv')

# 3月
sort_col = 'rtv'
df = price2.copy()
g_num = 5
hold_m = 3

rtv_ind, rtv = portfolio_sort(df,sort_col,g_num,hold_m,True)
rtv_ind.to_csv('.\\AMQI\\output\\indicator\\rtv3_indicators.csv')
rtv.to_csv('.\\AMQI\\output\\return\\rtv3.csv')

# 6月
sort_col = 'rtv'
df = price2.copy()
g_num = 5
hold_m = 6

rtv_ind, rtv = portfolio_sort(df,sort_col,g_num,hold_m,True)
rtv_ind.to_csv('.\\AMQI\\output\\indicator\\rtv6_indicators.csv')
rtv.to_csv('.\\AMQI\\output\\return\\rtv6.csv')

# 9月
sort_col = 'rtv'
df = price2.copy()
g_num = 5
hold_m = 9

rtv_ind, rtv = portfolio_sort(df,sort_col,g_num,hold_m,True)
rtv_ind.to_csv('.\\AMQI\\output\\indicator\\rtv9_indicators.csv')
rtv.to_csv('.\\AMQI\\output\\return\\rtv9.csv')

# 12月
sort_col = 'rtv'
df = price2.copy()
g_num = 5
hold_m = 12

rtv_ind, rtv = portfolio_sort(df,sort_col,g_num,hold_m,True)
rtv_ind.to_csv('.\\AMQI\\output\\indicator\\rtv12_indicators.csv')
rtv.to_csv('.\\AMQI\\output\\return\\rtv12.csv')

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 101.13it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:11<00:00,  1.07s/it]


In [77]:
rtv.drop('year_month',axis=1).mean()

return_long_eq      0.010771
return_short_eq    -0.005359
risk_free_month     0.001767
return_spread_eq    0.016130
return_long_mv      0.020155
return_short_mv     0.003631
return_spread_mv    0.016524
dtype: float64

#### pps

In [78]:
# 价格数据
price3 = price1.copy()
price3 = price3.drop(['qsba','esba'],axis=1)

# 合并indicator
pps = pd.read_csv('.\\AMQI\\processed\\pps1.csv')
pps = pps.rename(columns={pps.columns[0]:'code',
                          pps.columns[1]:'date',
                          pps.columns[2]:'pps'})
pps['date'] = pd.to_datetime(pps['date'])
pps['code'] = pps['code'].astype(str).str.zfill(6)
pps['year'] = pps.date.dt.year
pps['month'] = pps.date.dt.month
pps = pps.drop('date',axis=1)

price3 = pd.merge(price3,pps,on=['code','year','month'],how='left')
price3 = price3[(price3.date >= pd.to_datetime('20070601'))] # 确定时间

In [79]:
# 根据 rtv 计算分组
# 1月
sort_col = 'pps'
df = price3.copy()
g_num = 5
hold_m = 1

pps_ind, pps = portfolio_sort(df,sort_col,g_num,hold_m,True)
pps_ind.to_csv('.\\AMQI\\output\\indicator\\pps_indicators.csv')
pps.to_csv('.\\AMQI\\output\\return\\pps.csv')

# 3月
sort_col = 'pps'
df = price3.copy()
g_num = 5
hold_m = 3

pps_ind, pps = portfolio_sort(df,sort_col,g_num,hold_m,True)
pps_ind.to_csv('.\\AMQI\\output\\indicator\\pps3_indicators.csv')
pps.to_csv('.\\AMQI\\output\\return\\pps3.csv')

# 6月
sort_col = 'pps'
df = price3.copy()
g_num = 5
hold_m = 6

pps_ind, pps = portfolio_sort(df,sort_col,g_num,hold_m,True)
pps_ind.to_csv('.\\AMQI\\output\\indicator\\pps6_indicators.csv')
pps.to_csv('.\\AMQI\\output\\return\\pps6.csv')

# 9月
sort_col = 'pps'
df = price3.copy()
g_num = 5
hold_m = 9

pps_ind, pps = portfolio_sort(df,sort_col,g_num,hold_m,True)
pps_ind.to_csv('.\\AMQI\\output\\indicator\\pps9_indicators.csv')
pps.to_csv('.\\AMQI\\output\\return\\pps9.csv')

# 12月
sort_col = 'pps'
df = price3.copy()
g_num = 5
hold_m = 12

pps_ind, pps = portfolio_sort(df,sort_col,g_num,hold_m,True)
pps_ind.to_csv('.\\AMQI\\output\\indicator\\pps12_indicators.csv')
pps.to_csv('.\\AMQI\\output\\return\\pps12.csv')

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 109.04it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:11<00:00,  1.02s/it]


In [80]:
pps.drop('year_month',axis=1).mean()

return_long_eq      0.008850
return_short_eq    -0.006660
risk_free_month     0.001767
return_spread_eq    0.015510
return_long_mv      0.009434
return_short_mv     0.003554
return_spread_mv    0.005880
dtype: float64

#### Me

In [81]:
# 价格数据
price4 = price1.copy()
price4 = price4.drop(['qsba','esba'],axis=1)

# 合并indicator
me = price4[['code','year','month','market_value']]
me = me.rename(columns={'market_value':'me'})
me = me[me.month==6]

price4 = pd.merge(price4,me,on=['code','year','month'],how='left')
price4['me'] = price4.groupby(['code'])['me'].ffill() # 年更新，接下来一个财政年均使用相同的数值分组
price4 = price4[(price4.date >= pd.to_datetime('20070601'))] # 确定时间

In [82]:
# 根据 me 计算分组
sort_col = 'me'
df = price4.copy()
g_num = 5
hold_m = 1

me_ind, me = portfolio_sort(df,sort_col,g_num,hold_m,True)
me_ind.to_csv('.\\AMQI\\output\\indicator\\me_indicators.csv')
me.to_csv('.\\AMQI\\output\\return\\me.csv')

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 107.36it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [83]:
me.drop('year_month',axis=1).mean()

return_long_eq      0.007775
return_short_eq    -0.002204
risk_free_month     0.001843
return_spread_eq    0.009980
return_long_mv      0.018633
return_short_mv     0.005025
return_spread_mv    0.013608
dtype: float64

#### droa

In [84]:
# 价格数据
price5 = price1.copy()
price5 = price5.drop(['qsba','esba'],axis=1)

# 合并indicator
droa = pd.read_csv('.\\AMQI\\processed\\droa_final.csv',index_col=0)
droa['code'] = droa['code'].astype(str).str.zfill(6)

price5 = pd.merge(price5,droa,on=['code','year','month'],how='left')
price5['droa'] = price5.groupby(['code'])['droa'].ffill() # 年更新，接下来一个财政年均使用相同的数值分组
price5 = price5[(price5.date >= pd.to_datetime('20070601'))] # 确定时间

In [85]:
# 根据 droa 计算分组
sort_col = 'droa'
df = price5.copy()
g_num = 5
hold_m = 1

droa_ind, droa = portfolio_sort(df,sort_col,g_num,hold_m)
droa_ind.to_csv('.\\AMQI\\output\\indicator\\droa_indicators.csv')
droa.to_csv('.\\AMQI\\output\\return\\droa.csv')

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 77.22it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


#### Amihud & dfin

In [86]:
def data_modify(df):
    df = df.rename(columns={'anomaly_return_mv_rf':'return_spread_mv',
                            'anomaly_return_eq_rf':'return_spread_eq',
                            'return_short_mv':'return_long_mv',
                            'return_long_mv':'return_short_mv',
                            'return_short_eq':'return_long_eq',
                            'return_long_eq':'return_short_eq',}) # 调整列名

    # 调整数值
    df['return_short_mv'] = df['return_short_mv'] - df['risk_free_month']
    df['return_long_mv'] = df['return_long_mv'] - df['risk_free_month']
    df['return_short_eq'] = df['return_short_eq'] - df['risk_free_month']
    df['return_long_eq'] = df['return_long_eq'] - df['risk_free_month']
    df['return_spread_mv'] = df['return_long_mv'] - df['return_short_mv']
    df['return_spread_eq'] = df['return_long_eq'] - df['return_short_eq']
    
    # 调整日期
    df['date'] = pd.to_datetime(df['year_month'])
    df = df[df['date'] >= pd.to_datetime('20070601')]
    df = df.drop('date',axis=1)
    return df

In [87]:
ami = pd.read_excel('.\\AMQI\\pjoutput\\ami3.xlsx')
data_modify(ami).to_csv('.\\AMQI\\output\\return\\ami3.csv')

ami = pd.read_excel('.\\AMQI\\pjoutput\\ami6.xlsx')
data_modify(ami).to_csv('.\\AMQI\\output\\return\\ami6.csv')

ami = pd.read_excel('.\\AMQI\\pjoutput\\ami9.xlsx')
data_modify(ami).to_csv('.\\AMQI\\output\\return\\ami9.csv')

ami = pd.read_excel('.\\AMQI\\pjoutput\\ami12.xlsx')
data_modify(ami).to_csv('.\\AMQI\\output\\return\\ami12.csv')

dfin = pd.read_excel('.\\AMQI\\pjoutput\\dfin_standardized.xlsx')
data_modify(dfin).to_csv('.\\AMQI\\output\\return\\dfin.csv')

查看月度收益率

In [191]:
rtv_vw.mean()

year               2015.286432
month                 6.562814
long                  0.003875
short                 0.020761
risk_free_month       0.001846
spread               -0.016886
dtype: float64

In [166]:
qsba1 = qsba_vw.iloc[qsba_vw[(qsba_vw.year==2008) & (qsba_vw.month==7)].index[0]:
                     qsba_vw[(qsba_vw.year==2019) & (qsba_vw.month==7)].index[0],:]
qsba1.shape

(132, 6)

In [167]:
qsba_vw.mean()

year               2015.000000
month                 6.500000
long                  0.012871
short                 0.000087
risk_free_month       0.001857
spread                0.012784
dtype: float64

In [169]:
esba1 = esba_vw.iloc[esba_vw[(esba_vw.year==2008) & (esba_vw.month==7)].index[0]:
                     esba_vw[(esba_vw.year==2019) & (esba_vw.month==7)].index[0],:]
esba_vw.mean()

year               2014.626728
month                 6.479263
long                  0.016244
short                 0.001748
risk_free_month       0.001859
spread                0.014496
dtype: float64

In [46]:
a = pd.read_pickle('.\\AMQI\\raw\\price_month.pkl')

In [49]:
a[(a.code=='000001') & (a.date.dt.year==2006)]

,code,date,close_price,market_value,market,risk_free,total_value,return,market_value_weighted
123526,000001,2006-01-25,6.35,8949448.48,0.005655,0.000061,12355970.65,0.034202,8653482.47
124693,000001,2006-02-28,6.84,9640035.84,0.003393,0.000061,13309423.50,0.077165,8949448.48
125838,000001,2006-03-31,6.37,8977635.72,0.005847,0.000061,12394887.09,-0.068713,9640035.84
126954,000001,2006-04-28,7.88,11105772.28,0.017035,0.000061,15333078.53,0.237049,8977635.72
129161,000001,2006-06-30,7.56,10654776.46,-0.001261,0.000061,14710415.45,-0.040609,11105772.28
130222,000001,2006-07-31,6.72,9470912.40,-0.041587,0.000061,13075924.84,-0.111111,10654776.46
131407,000001,2006-08-31,7.23,10189687.01,0.004348,0.000068,14068294.14,0.075893,9470912.40
132653,000001,2006-09-29,8.16,11500393.63,0.011751,0.000068,15877908.74,0.128631,10189687.01
133916,000001,2006-10-31,9.55,13459406.77,0.011937,0.000068,18582601.52,0.170343,11500393.63
135195,000001,2006-11-30,12.65,17828428.86,0.024801,0.000068,24614650.18,0.324607,13459406.77
